<a href="https://colab.research.google.com/github/drpedrazas/compiladores-proyecto/blob/main/AnalizadorSintactico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import functools as ft
from google.colab import files


class Analizador:
  def __init__(self, file):
    self.grammar, self.non_terminals, self.terminals = self.read_grammar(file)
    print(self.terminals)
    self.per_prod_firsts = dict()
    self.firsts = self.all_primeros()
    self.follows_done = {i: False for i in self.non_terminals}
    self.follows = dict()
    self.genFollows()
    self.parsing_table = dict()
    self.genTable()

  def read_grammar(self, file):
    #returns a dict representing the gramamr and all non terminal symbols as a list and all terminal symbols as a list
    doc = list(filter(lambda x: x.strip() != "\n", open(file).readlines()))
    lines = [i.strip().replace(" ", "").split("->") for i in doc[:-1] if i.strip() not in {'\n',''}]
    return {i[0]: i[1].split("|") for i in lines}, [i[0] for i in lines], list(map(lambda x: x.strip(), doc[-1].split(" ")))
  
  def primero(self, x):
    if x in self.terminals:
      return {x}
    if x == "$":
      return {x}
    primero_x = set()
    self.per_prod_firsts[x] = list()
    for i in self.grammar[x]:
      void = True
      temp = set()
      for e in i:
        if "$" in self.primero(e):
          temp = temp.union(self.primero(e))
          continue
        else:
          void = False
          temp = temp.union(self.primero(e))
          break
      if not void and "$" in temp:
        temp.remove("$")
      self.per_prod_firsts[x].append(list(temp))
      primero_x = primero_x.union(temp)
    return primero_x

  def all_primeros(self):
    all_simbols = self.non_terminals + self.terminals
    return {i: self.primero(i) for i in all_simbols}

  def chain_primeros(self, x):
    return set(ft.reduce(lambda a, b: a.union(b), [self.firsts[i] for i in x]))

  def follows_fill(self, x):
    self.follows_done[x] = True
    followsTemp = [i for i in self.follows[x]]
    for s in self.follows[x]:
      if s in self.non_terminals:
        if not self.follows_done[s]:
          followsTemp += self.follows_fill(s)
          followsTemp.remove(s)
          if x in followsTemp:
            followsTemp.remove(x)
        else:
          followsTemp += list(self.follows[s])
          followsTemp.remove(s)
          if x in followsTemp:
            followsTemp.remove(x)
    self.follows[x] = set(followsTemp)
    return self.follows[x]

  def genFollows(self):
    for i in range(len(self.non_terminals)):
      entry = self.non_terminals[i]
      if i == 0:
        self.follows[entry] = {"$"}
      for rule in self.grammar[entry]:
        for j in range(len(rule)):
          if rule[j] not in self.non_terminals:
            continue
          else:
            if rule[j] not in self.follows:
              self.follows[rule[j]] = set()
            if(j != len(rule) - 1):
              self.follows[rule[j]] = self.follows[rule[j]].union(self.chain_primeros(rule[j+1:]))
              if("$" in self.follows[rule[j]]):
                self.follows[rule[j]].remove("$")
                self.follows[rule[j]].add(entry)
            else:
              if rule[j] != entry:
                self.follows[rule[j]].add(entry)
    for entry in self.follows:
      self.follows_fill(entry)

  def genTable(self):
      for element in self.non_terminals:
        self.parsing_table[element] = dict()
        if self.firsts[element] != set():
          index = 0
          for prod in self.per_prod_firsts[element]:
            for i in range(0, len(prod)):
              if prod[i] != "$":
                self.parsing_table[element][prod[i]] = self.grammar[element][index]
              else:
                for follow in self.follows[element]:
                  self.parsing_table[element][follow] = self.grammar[element][index]
            index += 1

 # def analisis():
 #   stack = ["@"]
 #   stack.append(self.non_terminals[0])
 #   curr = 0
 #   a = tokens[curr]
  #  top = stack[-1]


In [ ]:
ana = Analizador("/content/test.txt")
print(ana.terminals)
print(ana.grammar)
print(ana.firsts)
print(ana.follows)
print(ana.per_prod_firsts)
print(ana.parsing_table)

['+', '*', 'x', '$']


KeyError: ignored